>อันนี้คือแค่ลองรันดูแต่ละ cell ไม่ได้อยู่ใน production หลัก

แต่ที่หลักๆคือ **TTS.py**

In [1]:
!!pip install gTTS

['Requirement already satisfied: gTTS in /opt/homebrew/lib/python3.10/site-packages (2.5.4)',
 'Requirement already satisfied: requests<3,>=2.27 in /opt/homebrew/lib/python3.10/site-packages (from gTTS) (2.32.3)',
 'Requirement already satisfied: click<8.2,>=7.1 in /opt/homebrew/lib/python3.10/site-packages (from gTTS) (8.1.8)',
 'Requirement already satisfied: charset-normalizer<4,>=2 in /opt/homebrew/lib/python3.10/site-packages (from requests<3,>=2.27->gTTS) (3.4.1)',
 'Requirement already satisfied: idna<4,>=2.5 in /opt/homebrew/lib/python3.10/site-packages (from requests<3,>=2.27->gTTS) (3.10)',
 'Requirement already satisfied: urllib3<3,>=1.21.1 in /opt/homebrew/lib/python3.10/site-packages (from requests<3,>=2.27->gTTS) (2.3.0)',
 'Requirement already satisfied: certifi>=2017.4.17 in /opt/homebrew/lib/python3.10/site-packages (from requests<3,>=2.27->gTTS) (2025.1.31)']

In [35]:
from gtts import gTTS

In [27]:
thai_text = """
อะระหัง สัมมา สัมพุทโธ ภะคะวา อะระหัง สัมมา สัมพุทโธ ภะคะวา
พุทธัง ภาควันตัง อภิวาเทมิ พุทธัง ภาควันตัง อภิวาเทมิ
สวากขาโต ภะคะวะตาธัมโม ธัมมัง นะมะสาหมิ ธัมมัง นะมะสาหมิ
สุปฏิปันโน ภะคะวะโต สาวกสังโฆ สุปฏิปันโน ภะคะวะโต สาวกสังโฆ
สังฆัง นะมามิ สังฆัง นะมามิ
นะโม อมิตาภะ นะโม ตถาคตโต อรหโต สัมมาสัมพุทธัสสะ...
(ต่อได้เรื่อยๆ)
"""

In [39]:
import gtts

In [40]:
from gtts import gTTS

In [ ]:
print(gtts.lang.tts_langs())

TypeError: tts_langs() got an unexpected keyword argument 'tld'

In [42]:
gTTS.lang.tts_langs(tld='com')

AttributeError: type object 'gTTS' has no attribute 'lang'

In [28]:
tts = gTTS(text=thai_text, lang='th')

output_path = '/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output.mp3'
tts.save(output_path)

## PostProcess

In [10]:
from pydub import AudioSegment
from pydub.playback import play
import numpy as np

In [25]:
# Download audio
audio = AudioSegment.from_file("/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output.mp3")

# 2. ปรับความดังเบื้องต้น (ถ้าเสียงต้นฉบับเบาหรือดังเกินไป)
audio = audio.set_channels(1) # เพื่อความง่ายในการประมวลผล, แปลงเป็น mono
audio = audio.set_frame_rate(44100) # กำหนด frame rate มาตรฐาน

# 3. ลดความเร็วและปรับ Pitch อย่างละเอียดขึ้น (ใช้เทคนิคที่ซับซ้อนขึ้น)
# วิธีนี้พยายามรักษา pitch เมื่อลดความเร็ว แต่ก็ยังจำกัดใน Pydub
# ถ้าต้องการคุณภาพสูง ต้องใช้ Librosa/PyDub combined with external pitch shifters
slowed_audio_rate = int(audio.frame_rate * 0.9) # ลดความเร็วลง 30%
slowed_audio = audio._spawn(audio.raw_data, overrides={"frame_rate": slowed_audio_rate})
slowed_audio = slowed_audio.set_frame_rate(audio.frame_rate) # ตั้ง frame rate กลับ เพื่อรักษา pitch ให้ใกล้เคียง

# 4. เพิ่ม Reverb effect (เสียงก้อง) ให้เป็นธรรมชาติขึ้น
# การทำ reverb ด้วย pydub ให้ผลลัพธ์ที่จำกัด
# คุณอาจต้องลองใช้ค่า delay และ gain ที่หลากหลายขึ้น หรือพิจารณาใช้ไลบรารีอื่น
reverb_audio = slowed_audio
initial_delay = 300 # ms
decay_factor = 0.6 # ปัจจัยการลดทอนของแต่ละ echo (0-1)

# สร้างหลายๆ echo ที่ลดทอนลงไปเรื่อยๆ
for i in range(1, 5): # สร้าง 4 layers ของ echo
    delay = initial_delay * i
    gain = -20 * i * decay_factor # ลด gain ให้มากขึ้นตามจำนวน echo
    echo = slowed_audio.apply_gain(gain)
    reverb_audio = reverb_audio.overlay(echo, position=delay)

# 5. กรองเสียงให้ทุ้มขึ้นและชัดเจนขึ้น (ปรับ EQ)
# บทสวดมักจะมีเสียงย่านต่ำที่ชัดเจน แต่ก็ยังคงรายละเอียดของเสียงพูด
# ลองปรับ Low-pass filter ให้ออกไปในย่านที่สูงขึ้น เพื่อให้เสียงไม่ทึบเกินไป
# และ High-pass filter เพื่อตัดเสียงรบกวนต่ำๆ ที่ไม่จำเป็นออก
filtered_audio = reverb_audio.low_pass_filter(2500) # กรองความถี่สูงกว่า 2500 Hz (ให้เสียงทุ้มขึ้นแต่ยังชัด)
filtered_audio = filtered_audio.high_pass_filter(100) # เอาเสียงต่ำเกินไปออก (เพื่อให้เสียงไม่บวม)

# 6. เพิ่มความดังและลดเสียงที่ดังเกินไป
# บทสวดมักจะมีความดังที่สม่ำเสมอ
final_audio = filtered_audio.normalize() # ปรับความดังสูงสุดให้เป็น 0 dBFS
final_audio = final_audio.apply_gain(-3) # ลดเสียงลง 3 dB จากจุดสูงสุด เพื่อไม่ให้เสียงแตก

# 7. เล่นและบันทึกเสียง
final_audio.export("/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/monk_chant_improved.wav", format="wav")

print("เสียงสวดแบบพระถูกบันทึกเป็น monk_chant_improved.wav แล้ว")

เสียงสวดแบบพระถูกบันทึกเป็น monk_chant_improved.wav แล้ว


In [22]:
!!pip install librosa
!!pip install soundfile

['Requirement already satisfied: soundfile in /opt/homebrew/lib/python3.10/site-packages (0.13.1)',
 'Requirement already satisfied: cffi>=1.0 in /opt/homebrew/lib/python3.10/site-packages (from soundfile) (1.17.1)',
 'Requirement already satisfied: numpy in /opt/homebrew/lib/python3.10/site-packages (from soundfile) (2.0.2)',
 'Requirement already satisfied: pycparser in /opt/homebrew/lib/python3.10/site-packages (from cffi>=1.0->soundfile) (2.22)']

In [31]:
from pydub import AudioSegment

def change_pitch(audio_segment, octaves):
    new_sample_rate = int(audio_segment.frame_rate * (2.0 ** octaves))
    return audio_segment._spawn(audio_segment.raw_data, overrides={'frame_rate': new_sample_rate}).set_frame_rate(44100)

sound = AudioSegment.from_file("/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output.mp3")
lowered = change_pitch(sound, -0.4)  # -0.5 = ลง 1/2 octave (ใกล้เสียงผู้ชาย)

lowered.export("/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output_CC.mp3", format="mp3")


<_io.BufferedRandom name='/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output_CC.mp3'>

In [32]:
!!brew install sox

['==> Fetching downloads for: sox',
 '==> Downloading https://ghcr.io/v2/homebrew/core/sox/manifests/14.4.2_6',
 '==> Fetching dependencies for sox: mad and opusfile',
 '==> Downloading https://ghcr.io/v2/homebrew/core/mad/manifests/0.15.1b-2',
 '==> Fetching mad',
 '==> Downloading https://ghcr.io/v2/homebrew/core/mad/blobs/sha256:1facd8abee7e62bba7c7e445d2cbc5900dfe2dc7903be43a40b128c68f519b02',
 '==> Downloading https://ghcr.io/v2/homebrew/core/opusfile/manifests/0.12_1',
 '==> Fetching opusfile',
 '==> Downloading https://ghcr.io/v2/homebrew/core/opusfile/blobs/sha256:ad3b05a6931361ed1be9fd61d7c378d149071cf50bf0e7741df0799da481849a',
 '==> Fetching sox',
 '==> Downloading https://ghcr.io/v2/homebrew/core/sox/blobs/sha256:dd3179a8e7d8399ef404e8e19e199d7cb2d43bc7472cc6e5932777dd204ea20f',
 '==> Installing dependencies for sox: mad and opusfile',
 '==> Installing sox dependency: mad',
 '==> Downloading https://ghcr.io/v2/homebrew/core/mad/manifests/0.15.1b-2',
 'Already downloaded: /U

In [33]:
import os

# ตั้งชื่อไฟล์ input/output
input_path = "/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output.mp3"
temp_wav_path = "/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/output.wav"
output_path = "/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/output_mp3/voice_male_chant.wav"

# แปลง .mp3 เป็น .wav (sox แนะนำให้ใช้ wav)
os.system(f"sox '{input_path}' '{temp_wav_path}'")

# ใช้ sox ลด pitch ลง 500 cent (5 semitone) เพื่อให้เสียงทุ้มลงเหมือนเสียงพระสวด
# และใช้ highpass + lowpass filter กรองให้เสียงนุ่มลึก
os.system(f"sox '{temp_wav_path}' '{output_path}' pitch -500 highpass 80 lowpass 300")

print("✅ แปลงสำเร็จ: voice_male_chant.wav")


✅ แปลงสำเร็จ: voice_male_chant.wav


In [18]:
import os

os.remove('/Users/psoongsu/Desktop/MyCode/Stupid9/sht9-paliscript/TTS/monk_chant_improved.wav')

In [47]:
thai_text = """
อะระหัง สัมมา สัมพุทโธ ภะคะวา อะระหัง สัมมา สัมพุทโธ ภะคะวา
พุทธัง ภาควันตัง อภิวาเทมิ พุทธัง ภาควันตัง อภิวาเทมิ
สวากขาโต ภะคะวะตาธัมโม ธัมมัง นะมะสาหมิ ธัมมัง นะมะสาหมิ
สุปฏิปันโน ภะคะวะโต สาวกสังโฆ สุปฏิปันโน ภะคะวะโต สาวกสังโฆ
สังฆัง นะมามิ สังฆัง นะมามิ
นะโม อมิตาภะ นะโม ตถาคตโต อรหโต สัมมาสัมพุทธัสสะ...
(ต่อได้เรื่อยๆ)
"""

In [52]:
import http.client
import json # นำเข้าไลบรารีสำหรับทำงานกับ JSON

# --- ข้อมูลที่คุณต้องกรอก ---
YOUR_API_KEY = "Bearerb Y92rMtxZcQ4" # <--- **แทนที่ตรงนี้ด้วย API Key ของคุณจริงๆ**
SANSKRIT_CHART_TEXT = thai_text # <--- **ใส่บทสวดสันสกฤตของคุณตรงนี้**
# -------------------------

# เลือก endpoint สำหรับเสียงผู้ชาย (สามารถเลือก 'tts-sita' หรือ 'tts-ram' ได้)
# สำหรับตัวอย่างนี้เราจะใช้ 'tts-sita'
ENDPOINT = "/api/v1/tts-turbo/tts-sita"

conn = http.client.HTTPSConnection("api.celestiai.co")

# สร้าง payload ในรูปแบบ JSON
# API ต้องการ key ชื่อ "text" ที่มีค่าเป็นข้อความที่เราต้องการให้พูด
payload_data = {
    "text": SANSKRIT_CHART_TEXT
}
payload = json.dumps(payload_data) # แปลง Python dictionary ให้เป็น JSON string

headers = {
    'Content-Type': "application/json",
    'x-api-key': YOUR_API_KEY # **เพิ่ม API Key ของคุณใน Header นี้**
}

try:
    conn.request("POST", ENDPOINT, payload, headers)
    res = conn.getresponse()
    response_data = res.read()

    # ลอง decode เป็น JSON เพื่อดู response
    try:
        json_response = json.loads(response_data.decode("utf-8"))
        print("API Response (JSON):")
        print(json.dumps(json_response, indent=2, ensure_ascii=False)) # พิมพ์ JSON ให้สวยงาม

        # ถ้าสำเร็จ API จะคืนค่า URL ของไฟล์เสียงมาให้
        if "data" in json_response and "audio_url" in json_response["data"]:
            audio_url = json_response["data"]["audio_url"]
            print(f"\nเสียงบทสวดของคุณอยู่ที่ URL: {audio_url}")
            print("คุณสามารถคัดลอก URL นี้ไปเปิดในเว็บเบราว์เซอร์เพื่อฟังหรือดาวน์โหลดได้เลย")
        elif "error" in json_response:
            print(f"\nเกิดข้อผิดพลาดจาก API: {json_response['error']}")
        else:
            print("\nได้รับ response แต่ไม่พบ audio_url หรือข้อความ error ที่ชัดเจน")

    except json.JSONDecodeError:
        print("\nAPI Response (Raw Text - ไม่ใช่ JSON):")
        print(response_data.decode("utf-8")) # ถ้าไม่ใช่ JSON ก็พิมพ์เป็น text ไปเลย

except Exception as e:
    print(f"\nเกิดข้อผิดพลาดในการเชื่อมต่อหรือส่ง Request: {e}")

finally:
    conn.close() # ปิดการเชื่อมต่อเมื่อทำงานเสร็จสิ้น


API Response (Raw Text - ไม่ใช่ JSON):
NOT_FOUND
